# EU National Emission Ceilings (NEC) Directive Inventory

Data from 1990 to 2021

Source : https://www.eea.europa.eu/data-and-maps/data/ds_resolveuid/2BFB06C9-AB28-41EC-9808-576E32A36410

By : Robin Lotode

Setup : Download CSV file from Source link, add it in project/data/raw

Documentation : (from source file NEC_NFR19_2023_2023.xlsx)

National Emission Ceilings (NEC) Directive Inventory - NFR19 sector classification - feature catalogue

|Name |	Definition |
|---|---|
|Emissions  (float(8)) |	Emission value. |
|Country_code  (varchar(4)) |	International Country Code. Note: ISO 3166-1-Alpha-2 code elements |
|Country  (varchar(53)) |	Country name. |
|Pollutant_name  (varchar(20)) |	Short name of pollutant. Note: NH3, NMVOC, NOX, SO2 |
|Format_name  (varchar(100)) |	Name of guideline. Note: NFR19 sector classification |
|Sector_code  (varchar(15)) |	Sector code. Note: NFR19 sector classification |
|Parent_sector_code  (varchar(15)) |	Parent sector code. Note: NFR09 sector classification |
|Sector_name  (varchar(75)) |	Sector name. Note: NFR19 sector classification |
|Year  (varchar(4)) |	Annual data. Note: 1990-2021 |
|Unit  (varchar(40)) |	Emission unit. Note: Kilotonne (1000 tonnes) |
|Notation  (varchar(40)) |	Notation key. |


# Exploration

## Imports

In [1]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
import plotly
import plotly.express as px
import plotly.graph_objects as go

pd.options.plotting.backend = "plotly"
pd.set_option('expand_frame_repr', False)
# InteractiveShell.ast_node_interactivity = "all"


## Opening the dataset

In [2]:
df = pd.read_csv("../data/raw/NEC_NFR19_2023_23.06.27.csv", delimiter="\t")
df = df.rename(str.lower, axis='columns')
display(df.dtypes)
display(df.shape)
display(df.head())

country_code           object
country                object
pollutant_name         object
format_name            object
sector_code            object
parent_sector_code     object
sector_name            object
year                    int64
emissions             float64
unit                   object
notations              object
dtype: object

(4430070, 11)

,country_code,country,pollutant_name,format_name,sector_code,parent_sector_code,sector_name,year,emissions,unit,notations
0,EE,Estonia,SO2,NEC NFR14 sector classification,1A4bii,NATIONAL TOTAL,Residential: Household and gardening (mobile),2002,0.005180,Gg (1000 tonnes),NaN
1,MT,Malta,"Indeno (1,2,3-cd) Pyrene",NEC NFR14 sector classification,1A5a,NATIONAL TOTAL,Other stationary (including military),1997,NaN,t,NO
2,RO,Romania,BC,NEC NFR14 sector classification,2D3b,NATIONAL TOTAL,Road paving with asphalt,2021,0.008868,Gg (1000 tonnes),NaN
3,FR,France,PM2.5,NEC NFR14 sector classification,1A4bii,NATIONAL TOTAL,Residential: Household and gardening (mobile),1982,NaN,Gg (1000 tonnes),NR
4,DE,Germany,BC,NEC NFR14 sector classification,1A4cii,NATIONAL TOTAL,Agriculture/Forestry/Fishing: Off-road vehicle...,2005,3.093850,Gg (1000 tonnes),NaN


## Checking values

In [18]:
print("Unique values :")
df.isnull().sum()

df_dict = {}
for col in df.columns:
    if col != "emissions":
        print(f'{col} unique values : {len(list(df[col].unique()))}')
        df_dict[col] = df[col].value_counts(dropna=False)


Unique values :
country_code unique values : 28
country unique values : 28
pollutant_name unique values : 31
sector_code unique values : 139
parent_sector_code unique values : 2
sector_name unique values : 136
year unique values : 42
unit unique values : 5
notations unique values : 12


The column **format_name** has one single unique value "NEC NFR14 sector classification", so we can ignore it

In [4]:
df = df.drop("format_name", axis=1)

Let's see the **emissions** unit distribution :

In [5]:
for key, value in df_dict.items():
    if key in ["unit"]:
        print(key, ":", value, "\n")

unit : unit
t                   1729350
Gg (1000 tonnes)    1712520
TJNCV                617625
kg                   247050
g I-TEQ              123525
Name: count, dtype: int64 



**Notation keys** appear when the **Emissions** value is missing, explaining the reason why.

**Notation keys** : (from https://unfccc.int/files/national_reports/annex_i_ghg_inventories/reporting_requirements/application/pdf/crf_reporter_user_manual.pdf#page=74)
|Key | Meaning |
|---|---|
|NO | not occurring |
|NE | not estimated |
|NA | not applicable |
|IE | included elsewhere |
|C | Confidential |

Some **notation key** values aren't in the expected values [NO, NE, NR, IE, C] :

In [6]:
for key, value in df_dict.items():
    if key in ["notations"]:
        print(key, ":", value, "\n")
display(df[(df["notations"].notnull()) & (~df["notations"].isin(["NO", "NE", "NR", "IE", "C"]))])

notations : notations
NaN    2887851
NO      802444
NE      340031
NR      200782
IE      197660
??         563
C          451
N.          98
N/          64
N?          64
?.          61
Na           1
Name: count, dtype: int64 



,country_code,country,pollutant_name,sector_code,parent_sector_code,sector_name,year,emissions,unit,notations
7352,GR,Greece,PM2.5,1B2aiv,NATIONAL TOTAL,Fugitive emissions oil: Refining / storage,2017,NaN,Gg (1000 tonnes),??
15048,GR,Greece,NH3,1A4ai,NATIONAL TOTAL,Commercial/institutional: Stationary,2005,NaN,Gg (1000 tonnes),??
22490,GR,Greece,NOx,1B2aiv,NATIONAL TOTAL,Fugitive emissions oil: Refining / storage,2012,NaN,Gg (1000 tonnes),??
27226,LT,Lithuania,NOx,3Dc,NATIONAL TOTAL,Farm-level agricultural operations including s...,2001,NaN,Gg (1000 tonnes),??
32754,GR,Greece,Se,1A3ai(ii),NaN,International aviation cruise (civil),2012,NaN,t,N.
...,...,...,...,...,...,...,...,...,...,...
4403326,GR,Greece,NOx,1B2aiv,NATIONAL TOTAL,Fugitive emissions oil: Refining / storage,2017,NaN,Gg (1000 tonnes),??
4406197,GR,Greece,SO2,1B2aiv,NATIONAL TOTAL,Fugitive emissions oil: Refining / storage,2013,NaN,Gg (1000 tonnes),??
4414345,GR,Greece,Cu,1A3aii(ii),NaN,Domestic aviation cruise (civil),2012,NaN,t,N.
4418311,GR,Greece,TSP,1B2aiv,NATIONAL TOTAL,Fugitive emissions oil: Refining / storage,2021,NaN,Gg (1000 tonnes),??


The "parent_sector_code" field is either "NATIONAL TOTAL" or a missing value. 

In [7]:
display(df["parent_sector_code"].value_counts(dropna=False))
display(df[(df["country_code"] == "FR") & (df["year"] == 2020)]["parent_sector_code"].value_counts(dropna=False))

parent_sector_code
NATIONAL TOTAL    4167505
NaN                262565
Name: count, dtype: int64

parent_sector_code
NATIONAL TOTAL    3937
NaN                248
Name: count, dtype: int64

In [8]:
print("all pollutants", *df_dict["pollutant_name"].index, sep=" | ")
print("all sectors", *df_dict["sector_name"].index, sep=" | ")

all pollutants | SO2 | NH3 | PM2.5 | NMVOC | NOx | Biomass | Other Fuels | PM10 | Pb | Cd | Cu | HCB | PCBs | Hg | Se | Benzo(b) Fluoranthene | Benzo(a) Pyrene | Solid Fuels | As | Ni | Indeno (1,2,3-cd) Pyrene | CO | Liquid Fuels | PCDD/PCDF (dioxins/furans) | Zn | Cr | Gaseous Fuels | TSP | Total PAHs | BC | benzo(k) Fluoranthene
all sectors | NATIONAL TOTAL FOR COMPLIANCE | Residential: Household and gardening (mobile) | Agriculture/Forestry/Fishing: Stationary | Carbide production | Fugitive emissions oil: Refining / storage | Open burning of waste | Titanium dioxide production | Manure management - Dairy cattle  | Off-farm storage, handling and transport of bulk agricultural products | Industrial wastewater handling | Cement production | Manure management - Sheep | Other mineral products | Food and beverages industry  | Railways | Nickel production | National navigation (shipping) | Wood processing | Other product use | Domestic solvent use including fungicides | Degreasing | Quar

In [9]:
pollutants_df = pd.DataFrame(columns=["unit"])
pollutants_df.index.name = "pollutant_name"
for poll in df_dict["pollutant_name"].index:
    val_count = df[df["pollutant_name"] == poll]["unit"].value_counts()
    if len(val_count) != 1:
        print(poll, val_count)
    pollutants_df.loc[poll] = val_count.index[0]
print("pollutants and their unit")
display(pollutants_df)

pollutants and their unit


,unit
pollutant_name,
SO2,Gg (1000 tonnes)
NH3,Gg (1000 tonnes)
PM2.5,Gg (1000 tonnes)
NMVOC,Gg (1000 tonnes)
NOx,Gg (1000 tonnes)
Biomass,TJNCV
Other Fuels,TJNCV
PM10,Gg (1000 tonnes)
Pb,t


In [10]:
# sectors_df = pd.DataFrame(columns=["pollutant_name", "count"])
# sectors_df.index.name = "sector_name"
# for sector in df_dict["sector_name"].index[:2]:
#     val_count = df[df["sector_name"] == sector]["pollutant_name"].value_counts()
#     if sector == "NATIONAL TOTAL FOR COMPLIANCE":
#         display(val_count)
#     sectors_df.loc[sector] = [val_count, ]
# display(sectors_df)
sectors_df = df[(df["country"] == "France") & (df["year"] == 2000)].dropna(subset=["emissions"]).groupby(["sector_name", "pollutant_name"]).size()
print("emissions per sector")
display(sectors_df)

emissions per sector


sector_name                                           pollutant_name
Adipic acid production                                CO                1
                                                      NMVOC             1
                                                      NOx               1
Agriculture/Forestry/Fishing: National fishing        As                1
                                                      BC                1
                                                                       ..
Venting and flaring (oil, gas, combined oil and gas)  Zn                1
Wood processing                                       NMVOC             1
                                                      PM10              1
                                                      PM2.5             1
                                                      TSP               1
Length: 1414, dtype: int64

# Plotting

## Missing data visuals

In [11]:
missing_df = df.groupby(["year", "country"])["parent_sector_code"].value_counts(dropna=False)

years = missing_df.index.get_level_values("year").unique().values
year = 2020
fig = px.bar(
    x=missing_df[missing_df.index.get_level_values('year') == year].index.get_level_values(1),
    y=missing_df[missing_df.index.get_level_values('year') == year].values, 
    color=missing_df[missing_df.index.get_level_values('year') == year].index.get_level_values(2).isnull(), 
    title=f"parent_sector_code value counts per country in {year}",
    labels={"x": "Country", "y": "Count", "color": "parent_sector_code missing"},
    text_auto=".2s",
)
fig.update_traces(width=1)
fig.show()

In [12]:
missing_df_total = df.groupby(["year"])["parent_sector_code"].value_counts(dropna=False)
fig = px.bar(
    x=missing_df_total.index.get_level_values(0),
    y=missing_df_total.values, 
    color=missing_df_total.index.get_level_values(1).isnull(), 
    title=f"parent_sector_code value counts per country per year",
    labels={"x": "Year", "y": "Count", "color": "parent_sector_code missing"},
    text_auto=".2s",
)
fig.update_traces(width=1)
fig.show()


## Visuals

### Per pollutant by year in a specific country

In [13]:
pollutant_test = "PM2.5"
country = "Ireland"
data = df[(df["pollutant_name"] == pollutant_test) & (df["country"] == country)]

top_avg = data.sort_values(["emissions", "year"], ascending=[False, False], na_position="last")
# display(top_avg)

(top_avg
    .iloc[:10]
    .plot(kind="bar", barmode="group", x="year", y=["emissions"], color="sector_name", 
          title=f"{pollutant_test} emissions ({pollutants_df.loc[pollutant_test].values[0]}) per sector in {country}",
          labels={"year": "Year", "value": f"{pollutant_test} emissions ({pollutants_df.loc[pollutant_test].values[0]})", "sector_name": "Sector"})
)

In [14]:
for pollutant in pollutants_df.index[:3]:
  (
    df[(df["pollutant_name"] == pollutant) & (df["country"] == country)]
      .sort_values(["emissions", "year"], ascending=[False, False], na_position="last")
      .iloc[:10]
      .plot(kind="bar", barmode="group", x="year", y=["emissions"], color="sector_name", 
        title=f"{pollutant} emissions ({pollutants_df.loc[pollutant].values[0]}) per sector in {country}",
        labels={"year": "Year", "value": f"{pollutant} emissions ({pollutants_df.loc[pollutant].values[0]})", "sector_name": "Sector"})
      .show()
  )

### Per pollutant unit by year

In [15]:
selected_year_test = 2000
unit_test = "kg"

test_df = (
    df[(df["unit"] == unit_test) & (df["year"] == selected_year_test)]
    .groupby(["country", "year"])["emissions"]
    .sum()
    .sort_values(ascending=False, na_position="last")
    .drop("EU27", level=0)
)
# display(test_df)
# display(test_df.describe())

In [16]:
fig = px.bar(barmode="group",
       x=test_df.index.get_level_values(0),
       y=test_df.values, 
       color=test_df.index.get_level_values(0),
       title=f"emissions in {unit_test} per country in {selected_year_test}",
       labels={"x": "Country", "y": f"Emissions ({unit_test})", "color": "Country"},
       text_auto=".2s",
)
fig.update_traces(width=1)

In [20]:
print(f"pollutant units : {pollutants_df['unit'].unique()}")
selected_year = 2000
for unit in pollutants_df["unit"].unique()[:]:
    test_df = (
        df[(df["unit"] == unit) & (df["year"] == selected_year)]
        .groupby(["country", "year"])["emissions"]
        .sum()
        .sort_values(ascending=False, na_position="last")
        .drop("EU27", level=0)
    )
    if len(test_df) > 0:
        fig = px.bar(barmode="group",
            x=test_df.index.get_level_values(0),
            y=test_df.values, 
            color=test_df.index.get_level_values(0), 
            title=f"emissions in {unit} per country in {selected_year}",
            labels={"x": "Country", "y": f"Emissions ({unit})", "color": "Country"},
            text_auto=".2s",

            )
        fig.update_traces(width=1)
        fig.show()
    else:
        print(f"no emissions in {unit} per country in {selected_year}")

pollutant units : ['Gg (1000 tonnes)' 'TJNCV' 't' 'kg' 'g I-TEQ']


# Remarks

## Dataset anomalies :

- Some **notation key** values seem to be wrong : 
    | Key | Count |
    |---|---|
    |NO  |  802444|
    |NE  |  340031|
    |NR  |  200782|
    |IE  |  197660|
    |??  |     563|
    |C   |     451|
    |N.  |      98|
    |N/  |      64|
    |N?  |      64|
    |?.  |      61|
    |Na  |       1|  
    
    ["??", "N.", "N/", "N?", "?.", "Na"] seem to be misinputs.


## Remarks : 

- **Pollutants** have different **units**, making comparisons between **pollutants** of different **units** difficult / impossible.
- EU27 seems to represent the sum of the **emissions** of each country of the EU (no UK).
- A **sector** called "NATIONAL TOTAL FOR COMPLIANCE often appears in **emission sectors**, seems to represent the sum of all **emissions** for a **sector** but very unreliable : sometimes is here without another **emission**, sometimes the other way around.

## notes : 
- observer tendances
- regarder l'impact de mesures européennes prises
- regarder régulations, expliquer les émissions tombées à 0
- croiser datasets, générer un dataset avec features pour ML en sélectionnant les colonnes les + importantes
- étudier les caractéristiques des polluants, est-il obligatoire de publier
- librairie missingno pour explorer les valeurs manquantes
